In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from math import sqrt
import xgboost

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
%matplotlib inline

In [12]:
train_df = pd.read_csv("train.csv", low_memory=False)
test_df = pd.read_csv("test.csv", low_memory=False)
data = pd.read_csv("data_set.csv", low_memory = False)

In [13]:
train_df = train_df[['id','precio']]
train_df['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [14]:
data.shape

(300000, 12)

In [15]:
test_df.shape

(60000, 22)

In [16]:
train_df.shape

(240000, 2)

In [17]:
features = pd.merge(train_df, data, on='id', how='inner')
features = features.fillna(0)
features['precio'].value_counts()

1500000.0    3041
2500000.0    3036
1200000.0    2909
3500000.0    2726
1300000.0    2480
             ... 
2704600.0       1
1352250.0       1
938260.0        1
2704460.0       1
2327000.0       1
Name: precio, Length: 15520, dtype: int64

In [18]:
labels = features['precio']
features = features.drop(['id','precio'], axis=1)
x_train, x_test, y_train, y_test = train_test_split(features, labels, test_size=0.25)
print("Train: ",len(x_train),"Test: ",len(x_test))

Train:  180000 Test:  60000


In [19]:
set_test = pd.merge(data, test_df[['id']],on = "id", how = "inner")
ids = set_test["id"]
set_test = set_test.drop(columns=['id'])

In [20]:
set_test = set_test.fillna(0)

In [21]:
set_test.head()

,habitaciones,garages,banos,anio_x,gimnasio,usosmultiples,piscina,anio_y,qty_of_extras,metroscubiertos,metrostotales
0,3.0,0.0,4.0,2013,0.0,0.0,0.0,2013,0.0,300.0,0.0
1,1.0,1.0,1.0,2015,0.0,0.0,0.0,2015,0.0,67.0,67.0
2,2.0,1.0,2.0,2015,0.0,0.0,0.0,2015,0.0,87.0,100.0
3,2.0,2.0,2.0,2015,0.0,0.0,0.0,2015,0.0,86.0,86.0
4,2.0,1.0,1.0,2013,0.0,0.0,0.0,2013,0.0,80.0,76.0


In [24]:
xgb = xgboost.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
                           colsample_bytree=1, max_depth=7)
xgb.fit(x_train,y_train)

preds_kaggel_knn = xgb.predict(x_test)
print(mean_absolute_error(preds_kaggel_knn,y_test))

[22:19:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
948730.1112869792


In [25]:
df_kaggel_knn = pd.DataFrame(preds_kaggel_knn)
df_kaggel_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 1 columns):
0    60000 non-null float32
dtypes: float32(1)
memory usage: 234.5 KB


In [26]:
df_kaggel_knn = pd.DataFrame(df_kaggel_knn)

In [27]:
df_kaggel_knn.head()

,0
0,2.071595e+06
1,6.502949e+05
2,3.776195e+06
3,2.001079e+06
4,3.108394e+06


In [28]:
# df_kaggel_knn = df_kaggel_knn.to_frame()# df_kagg 
df_kaggel_knn["id"] = ids
df_kaggel_knn.rename(columns = {0: 'target'},inplace = True) 
df_kaggel_knn = df_kaggel_knn[['id','target']]

In [29]:
df_kaggel_knn.head()

,id,target
0,4941,2.071595e+06
1,51775,6.502949e+05
2,115253,3.776195e+06
3,299321,2.001079e+06
4,173570,3.108394e+06


In [30]:
df_kaggel_knn.to_csv('ResultadosXGB.csv',index=False)

In [31]:
df_kaggel_knn.shape

(60000, 2)

In [32]:
df_kaggel_knn.head()

,id,target
0,4941,2.071595e+06
1,51775,6.502949e+05
2,115253,3.776195e+06
3,299321,2.001079e+06
4,173570,3.108394e+06
